В данной задаче необходимо провести взлом шифра Вернама по 3 шифротекстам. Для начала загрузим шифротексты

In [1]:
import base64

texts = []
texts_paths = [
    "encrypted_texts/text1.txt",
    "encrypted_texts/text2.txt",
    "encrypted_texts/text3.txt",
]

for text in texts_paths:
    with open(text, "r", encoding='utf-8') as f:
        data = f.read().strip().strip("b'").strip("'")
        decoded_data = base64.b64decode(data)
        texts.append(decoded_data)

Узнаем длину текстов

In [2]:
print(len(texts[0]), len(texts[1]), len(texts[2]))

19530 19530 19530


In [3]:
text_length = len(texts[0])

Также зафиксируем множество допустимых символов для перебора: латинские буквы обоих регистров, знаки препинания, символ табуляции и символ перевода новой строки

В данный список можно было добавить также цифры, но так как комбинации цифр тяжело зарегламентировать (числа могут быть какие угодно), то это сильно усложнит процесс взлома, поэтому приходится делать допущение, что в наших текстах нет цифр

In [4]:
from string import ascii_letters
symbols_set = list(ascii_letters) + [" ", ".", ",", "?", "!", "-", ":", ";", "'", "_", "\"", "\t", "\n"]

Для взлома шифра используется свойство: 
$$c_1 = m_1 \oplus k, c_2 = m_2 \oplus k \Rightarrow c_1 \oplus c_2 = m_1 \oplus m_2$$

Данное свойство поможет при расшифровке исходных текстов следующим образом: мы сможем понимать, где совпадают биты исходных текстов, а где они различаются. Таким образом можно составлять и перебирать триплеты символов (тройка $T_i$ - три стоящих на i-х местах символа в трёх шифротекстах)

Для взлома шифра придется использовать перебор различных вариантов комбинаций и последовательностей символов - для того, чтобы немного облегчить данный процесс, совершим несколько предварительных действий

1) Для начала загрузим словарь английских слов, чтобы можно было отсекать варианты с бессмысленными последовательностями букв. Для этого можно использовать список слов из библиотеки Natural Language Toolkit

In [5]:
import nltk
from nltk.corpus import words
nltk.download('words')

english_vocabulary = set([str.lower(word) for word in words.words()])

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\79523\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


2) теперь составим возможные триплеты с помощью попарных XOR-ов и свойства, указанного выше. Для этого нам достаточно знать попарные XOR-ы между 1 и 2 текстами, а также между 1 и 3 текстами.

In [6]:
paired_xors = []

xors12 = bytes(a ^ b for (a, b) in zip(texts[0], texts[1]))
paired_xors.append(xors12)
xors13 = bytes(a ^ b for (a, b) in zip(texts[0], texts[2]))
paired_xors.append(xors13)

In [7]:
def triplets_for_one_position(symbols_set, xors):
    triplets = []
    for symbol in symbols_set:
        triplet = []
        triplet.append(symbol)
        valid_triplet = True
        for xor in xors:
            xor_symbol = ord(symbol) ^ xor
            if chr(xor_symbol) in symbols_set:
                triplet.append(chr(xor_symbol))
            else:
                valid_triplet = False
                break
        if valid_triplet:
            triplets.append(triplet)
    return triplets

In [8]:
triplets_for_all_text = []
for i in range(text_length):
    xors = bytes([xors12[i], xors13[i]])
    triplets_on_i_pos = triplets_for_one_position(symbols_set, xors)
    triplets_for_all_text.append(triplets_on_i_pos)

Пример возможных триплетов для первой позиции в текстах

In [9]:
triplets_for_all_text[0]

[['b', 'x', 'r'],
 ['c', 'y', 's'],
 ['h', 'r', 'x'],
 ['i', 's', 'y'],
 ['j', 'p', 'z'],
 ['q', 'k', 'a'],
 ['r', 'h', 'b'],
 ['s', 'i', 'c'],
 ['t', 'n', 'd'],
 ['u', 'o', 'e'],
 ['v', 'l', 'f'],
 ['w', 'm', 'g'],
 ['x', 'b', 'h'],
 ['y', 'c', 'i'],
 ['B', 'X', 'R'],
 ['C', 'Y', 'S'],
 ['E', '_', 'U'],
 ['H', 'R', 'X'],
 ['I', 'S', 'Y'],
 ['J', 'P', 'Z'],
 ['O', 'U', '_'],
 ['Q', 'K', 'A'],
 ['R', 'H', 'B'],
 ['S', 'I', 'C'],
 ['T', 'N', 'D'],
 ['U', 'O', 'E'],
 ['V', 'L', 'F'],
 ['W', 'M', 'G'],
 ['X', 'B', 'H'],
 ['Y', 'C', 'I'],
 ['_', 'E', 'O']]

Длина самого большого списка триплетов

In [10]:
len(max(triplets_for_all_text, key=len))

65

Длина самого маленького списка триплетов

In [11]:
len(min(triplets_for_all_text, key=len))

5

3) отсортируем списки триплетов для каждой позиции таким образом, чтобы первыми перебирать самые вероятные - таким образом есть шанс достаточно быстро найти искомые тексты

Для этого нам нужно определить частоты символов в английском языке - к сожалению, в открытых источниках тяжело найти готовое распределение частот, тем более для букв разного регистра или специальных символов, поэтому эту работу приходится делать самостоятельно следующим образом - взять какой-нибудь большой текст (например книгу с Project Guthenberg, спасибо за идею https://math.stackexchange.com/questions/3923188/does-there-exist-english-letter-frequency-charts-based-on-the-assumed-age-of-the ) и приблизительно определить частоты на его основе

In [12]:
# загружаем "War and Peace"
with open("resources/pg2600.txt","r", encoding='utf-8') as f:
    freq_text = f.read()

In [13]:
# с помощью коллекции Counter составляем словарь с количеством вхождений символов в текст
from collections import Counter

cnt = Counter(freq_text)
freq_text_len = len(freq_text)

In [14]:
# далее обрабатываем полученный Counter-объект - в новый словарь добавляем частоты, деля значения на длину текста
# после этого добавляем символы, которые не попали в исходный текст - даем им минимальную вероятность из имеющихся
possible_symbols_dict = {}
for item in cnt.items():
    if item[0] in symbols_set:
        possible_symbols_dict[item[0]] = item[1]/freq_text_len
        
for sym in symbols_set:
    if sym not in possible_symbols_dict.keys():
        possible_symbols_dict[sym] = min(possible_symbols_dict.values())

Сам словарь частот выглядит следующим образом

In [15]:
possible_symbols_dict

{'T': 0.0021099412624574834,
 'h': 0.05051651733900776,
 'e': 0.09646738204312956,
 ' ': 0.16023439247633983,
 'P': 0.0020182316275547795,
 'r': 0.04504523205945759,
 'o': 0.05839208223627748,
 'j': 0.0007023842646095617,
 'c': 0.018438903722853485,
 't': 0.06803119862633838,
 'G': 0.00040370829148048473,
 'u': 0.01986350136232184,
 'n': 0.055946905078668896,
 'b': 0.009619597109659317,
 'g': 0.015499547958252287,
 'B': 0.001116936600757596,
 'k': 0.005960196779132836,
 'f': 0.016405490432764135,
 'W': 0.0008932394507584325,
 'a': 0.06077374427511899,
 'd': 0.0360270146840758,
 '\n': 0.020460233648884704,
 'i': 0.0509065931171919,
 's': 0.04954272212786182,
 'y': 0.013936766206734586,
 'w': 0.017458292255059366,
 'U': 8.985065581683851e-05,
 'S': 0.0009267010743040137,
 'm': 0.018068347225070936,
 'p': 0.012085532867060067,
 'l': 0.029684797702797576,
 'v': 0.008045661483626423,
 '.': 0.009567855525102724,
 'Y': 0.00039193475727000243,
 ',': 0.01235849296283467,
 '-': 0.000548708660178

Для того, чтобы хранить вероятности триплетов в словаре, удобно ввести для триплетов некий идентификатор, т.к. использовать в качестве ключа кортеж с символами, или строку из символов, может быть не очень удобно из-за различий в порядке символов в триплетах

In [16]:
def create_triplet_id(triplet) -> int:
    ords = [ord(x) for x in triplet]
    ords.sort()
    triplet_id = ords[0] * 256 ** 2 + ords[1] * 256 + ords[2]
    return triplet_id

Словарь вероятностей триплетов

In [17]:
triplets_dict = {}
for i in range(len(symbols_set)):
    for j in range(i, len(symbols_set)):
        for k in range(j, len(symbols_set)):
            triplet = [symbols_set[i], symbols_set[j], symbols_set[k]]
            triplet_id = create_triplet_id(triplet)
            probability = possible_symbols_dict[symbols_set[i]] * possible_symbols_dict[symbols_set[j]] * possible_symbols_dict[symbols_set[k]]
            triplets_dict.update({triplet_id:probability})

Теперь можно запустить процесс перебора троек и составления читабельного текста. Для проверки читабельности были реализованы различные проверки в модуле tools.py, объединенные в одну функцию word_validation

Как только количество провалидированных символов в тексте достигнет 20, можно будет выводить текущие дешифруемые тексты, чтобы оценить их читабельность и работу алгоритма перебора

In [18]:
from tools import word_validation

In [19]:
possible_decryptions = []
init_triplets = triplets_for_all_text[0]
init_triplets.sort(key=lambda triplet: triplets_dict[create_triplet_id(triplet)])
for triplet in init_triplets:
    possible_decryptions.append({"triplet": triplet, "position": 0, "text1": "", "text2": "", "text3": ""})
    
while possible_decryptions:
    current = possible_decryptions.pop()

    current_text1 = current["text1"] + current["triplet"][0]
    valid1 = word_validation(current_text1, english_vocabulary)

    current_text2 = current["text2"] + current["triplet"][1]
    valid2 = word_validation(current_text2, english_vocabulary)

    current_text3 = current["text3"] + current["triplet"][2]
    valid3 = word_validation(current_text3, english_vocabulary)
    
    print(f'Текст 1: {current_text1}')
    print(f'Текст 2: {current_text2}')
    print(f'Текст 3: {current_text3}')
    print('-----------------------------------------')

    if all([valid1, valid2, valid3]):
        new_position = current["position"] + 1
        next_triplets = triplets_for_all_text[new_position]
        next_triplets.sort(key=lambda triplet: triplets_dict[create_triplet_id(triplet)])
        for triplet in next_triplets:
            possible_decryptions.append({"triplet": triplet, "position": new_position, "text1": current_text1, "text2": current_text2, "text3": current_text3})

    else:
        continue

Текст 1: t
Текст 2: n
Текст 3: d
-----------------------------------------
Текст 1: to
Текст 2: nt
Текст 3: dn
-----------------------------------------
Текст 1: th
Текст 2: ns
Текст 3: di
-----------------------------------------
Текст 1: th 
Текст 2: nsc
Текст 3: diu
-----------------------------------------
Текст 1: th,
Текст 2: nso
Текст 3: diy
-----------------------------------------
Текст 1: th!
Текст 2: nsb
Текст 3: dit
-----------------------------------------
Текст 1: th:
Текст 2: nsy
Текст 3: dio
-----------------------------------------
Текст 1: th:e
Текст 2: nsyr
Текст 3: dio.
-----------------------------------------
Текст 1: th:g
Текст 2: nsyp
Текст 3: dio,
-----------------------------------------
Текст 1: th:t
Текст 2: nsyc
Текст 3: dio?
-----------------------------------------
Текст 1: th:,
Текст 2: nsy;
Текст 3: diog
-----------------------------------------
Текст 1: th:p
Текст 2: nsyg
Текст 3: dio;
-----------------------------------------
Текст 1: th:;
Текст 2: ns

Текст 1: tr:;
Текст 2: niy,
Текст 3: dsop
-----------------------------------------
Текст 1: tr:A
Текст 2: niyV
Текст 3: dso

-----------------------------------------
Текст 1: tr:f
Текст 2: niyq
Текст 3: dso-
-----------------------------------------
Текст 1: tr:q
Текст 2: niyf
Текст 3: dso:
-----------------------------------------
Текст 1: tr:-
Текст 2: niy:
Текст 3: dsof
-----------------------------------------
Текст 1: tr::
Текст 2: niy-
Текст 3: dsoq
-----------------------------------------
Текст 1: tr:B
Текст 2: niyU
Текст 3: dso	
-----------------------------------------
Текст 1: tro
Текст 2: ni,
Текст 3: ds:
-----------------------------------------
Текст 1: troe
Текст 2: ni,r
Текст 3: ds:.
-----------------------------------------
Текст 1: trog
Текст 2: ni,p
Текст 3: ds:,
-----------------------------------------
Текст 1: trot
Текст 2: ni,c
Текст 3: ds:?
-----------------------------------------
Текст 1: tro,
Текст 2: ni,;
Текст 3: ds:g
-------------------------------------

Текст 1: tt,f
Текст 2: nooq
Текст 3: duy-
-----------------------------------------
Текст 1: tt,q
Текст 2: noof
Текст 3: duy:
-----------------------------------------
Текст 1: tt,-
Текст 2: noo:
Текст 3: duyf
-----------------------------------------
Текст 1: tt,:
Текст 2: noo-
Текст 3: duyq
-----------------------------------------
Текст 1: tt,B
Текст 2: nooU
Текст 3: duy	
-----------------------------------------
Текст 1: tt!
Текст 2: nob
Текст 3: dut
-----------------------------------------
Текст 1: tt!e
Текст 2: nobr
Текст 3: dut.
-----------------------------------------
Текст 1: tt!g
Текст 2: nobp
Текст 3: dut,
-----------------------------------------
Текст 1: tt!t
Текст 2: nobc
Текст 3: dut?
-----------------------------------------
Текст 1: tt!,
Текст 2: nob;
Текст 3: dutg
-----------------------------------------
Текст 1: tt!p
Текст 2: nobg
Текст 3: dut;
-----------------------------------------
Текст 1: tt!;
Текст 2: nob,
Текст 3: dutp
-------------------------------------

Текст 1: tuyp
Текст 2: nn:g
Текст 3: dt,;
-----------------------------------------
Текст 1: tuy;
Текст 2: nn:,
Текст 3: dt,p
-----------------------------------------
Текст 1: tuyA
Текст 2: nn:V
Текст 3: dt,

-----------------------------------------
Текст 1: tuyf
Текст 2: nn:q
Текст 3: dt,-
-----------------------------------------
Текст 1: tuyq
Текст 2: nn:f
Текст 3: dt,:
-----------------------------------------
Текст 1: tuy-
Текст 2: nn::
Текст 3: dt,f
-----------------------------------------
Текст 1: tuy:
Текст 2: nn:-
Текст 3: dt,q
-----------------------------------------
Текст 1: tuyB
Текст 2: nn:U
Текст 3: dt,	
-----------------------------------------
Текст 1: tu-
Текст 2: nnn
Текст 3: dtx
-----------------------------------------
Текст 1: tu;
Текст 2: nnx
Текст 3: dtn
-----------------------------------------
Текст 1: tu'
Текст 2: nnd
Текст 3: dtr
-----------------------------------------
Текст 1: tu"
Текст 2: nna
Текст 3: dtw
-----------------------------------------
Текс

Текст 1: tn"q
Текст 2: nuaf
Текст 3: dow:
-----------------------------------------
Текст 1: tn"-
Текст 2: nua:
Текст 3: dowf
-----------------------------------------
Текст 1: tn":
Текст 2: nua-
Текст 3: dowq
-----------------------------------------
Текст 1: tn"B
Текст 2: nuaU
Текст 3: dow	
-----------------------------------------
Текст 1: tnn
Текст 2: nu-
Текст 3: do;
-----------------------------------------
Текст 1: tn

Текст 2: nuI
Текст 3: do_
-----------------------------------------
Текст 1: tnx
Текст 2: nu;
Текст 3: do-
-----------------------------------------
Текст 1: tl
Текст 2: nw
Текст 3: dm
-----------------------------------------
Текст 1: tm
Текст 2: nv
Текст 3: dl
-----------------------------------------
Текст 1: tw
Текст 2: nl
Текст 3: dv
-----------------------------------------
Текст 1: tw 
Текст 2: nlc
Текст 3: dvu
-----------------------------------------
Текст 1: tw,
Текст 2: nlo
Текст 3: dvy
-----------------------------------------
Текст 1: tw!
Текст 2: nlb

...

Текст 1: in the fire, with her pretty hair flute.T
Текст 2: successful pupil was so great and consa"E
Текст 3: you? Yes. Well, there _is_ company thec.

-----------------------------------------
Текст 1: in the fire, with her pretty hair flute.y
Текст 2: successful pupil was so great and consa"h
Текст 3: you? Yes. Well, there _is_ company thec.'
-----------------------------------------
Текст 1: in the fire, with her pretty hair flute.W
Текст 2: successful pupil was so great and consa"F
Текст 3: you? Yes. Well, there _is_ company thec.	
-----------------------------------------
Текст 1: in the fire, with her pretty hair fluteT
Текст 2: successful pupil was so great and consaX
Текст 3: you? Yes. Well, there _is_ company thecT
-----------------------------------------
Текст 1: in the fire, with her pretty hair flute!
Текст 2: successful pupil was so great and consa-
Текст 3: you? Yes. Well, there _is_ company thec!
-----------------------------------------
Текст 1: in the fire, with her 

Текст 1: in the fire, with her pretty hair flutteringr
Текст 2: successful pupil was so great and conspicuou!
Текст 3: you? Yes. Well, there _is_ company there, I  
-----------------------------------------
Текст 1: in the fire, with her pretty hair fluttering!
Текст 2: successful pupil was so great and conspicuour
Текст 3: you? Yes. Well, there _is_ company there, I s
-----------------------------------------
Текст 1: in the fire, with her pretty hair fluttering;
Текст 2: successful pupil was so great and conspicuouh
Текст 3: you? Yes. Well, there _is_ company there, I i
-----------------------------------------
Текст 1: in the fire, with her pretty hair fluttering:
Текст 2: successful pupil was so great and conspicuoui
Текст 3: you? Yes. Well, there _is_ company there, I h
-----------------------------------------
Текст 1: in the fire, with her pretty hair fluttering?
Текст 2: successful pupil was so great and conspicuoul
Текст 3: you? Yes. Well, there _is_ company there, I m
-------

Текст 1: in the fire, with her pretty hair flutteriR
Текст 2: successful pupil was so great and conspicuS
Текст 3: you? Yes. Well, there _is_ company there, u
-----------------------------------------
Текст 1: in the fire, with her pretty hair flutteriF
Текст 2: successful pupil was so great and conspicuG
Текст 3: you? Yes. Well, there _is_ company there, a
-----------------------------------------
Текст 1: in the fire, with her pretty hair flutteriT
Текст 2: successful pupil was so great and conspicuU
Текст 3: you? Yes. Well, there _is_ company there, s
-----------------------------------------
Текст 1: in the fire, with her pretty hair flutteriU
Текст 2: successful pupil was so great and conspicuT
Текст 3: you? Yes. Well, there _is_ company there, r
-----------------------------------------
Текст 1: in the fire, with her pretty hair flutteriD
Текст 2: successful pupil was so great and conspicuE
Текст 3: you? Yes. Well, there _is_ company there, c
-------------------------------------

Текст 1: in the fire, with her pretty hair flutte?v
Текст 2: successful pupil was so great and conspi.j
Текст 3: you? Yes. Well, there _is_ company therea?
-----------------------------------------
Текст 1: in the fire, with her pretty hair flutte?k
Текст 2: successful pupil was so great and conspi.w
Текст 3: you? Yes. Well, there _is_ company therea"
-----------------------------------------
Текст 1: in the fire, with her pretty hair flutte?'
Текст 2: successful pupil was so great and conspi.;
Текст 3: you? Yes. Well, there _is_ company therean
-----------------------------------------
Текст 1: in the fire, with her pretty hair flutte?;
Текст 2: successful pupil was so great and conspi.'
Текст 3: you? Yes. Well, there _is_ company therear
-----------------------------------------
Текст 1: in the fire, with her pretty hair flutte?C
Текст 2: successful pupil was so great and conspi._
Текст 3: you? Yes. Well, there _is_ company therea

-----------------------------------------
Текст 1: i

KeyboardInterrupt: 

Перебор затянулся, но имеющегося результата, благодаря характерной комбинации 'successful pupil was so great', оказалось достаточно, чтобы найти один из предполагаемых исходных текстов

2-й текст (начало) - successful pupil was so great and conspicuous for that day that on the spot every scholars heart was fired with a fresh ambition that often lasted a couple of weeks. It is possible that Toms mental stomach had never really hungered for one of those prizes, but unquestionably his entire being had for many a day longed for the glory and the eclat that came with it. In due course the superintendent stood up in front of the pulpit, with a closed hymn-book in his hand and his forefinger inserted between its leaves, and commanded attention. When a Sunday-school superintendent makes his customary little speech, a hymn-book in the hand is as necessary as is the inevitable sheet of music in the hand of a singer who stands forward on the platform and sings a solo at a concertthough why, is a mystery: for neither the hymn-book nor the sheet of music is ever referred to by the sufferer.

Источник: Приключения Тома Сойера (глава 4), Марк Твен

С помощью него попытаемся найти ключ и расшифровать оставшиеся тексты (по крайней мере первые ~5000 символов)

In [20]:
#text2 = "successful pupil was so great and conspicuous for that day that on the spot every scholars heart was fired with a fresh ambition that often lasted a couple of weeks. It is possible that Toms mental stomach had never really hungered for one of those prizes, but unquestionably his entire being had for many a day longed for the glory and the eclat that came with it. In due course the superintendent stood up in front of the pulpit, with a closed hymn-book in his hand and his forefinger inserted between its leaves, and commanded attention. When a Sunday-school superintendent makes his customary little speech, a hymn-book in the hand is as necessary as is the inevitable sheet of music in the hand of a singer who stands forward on the platform and sings a solo at a concertthough why, is a mystery: for neither the hymn-book nor the sheet of music is ever referred to by the sufferer."

with open("text2first5000syms.txt", 'r', encoding="utf-8") as f:
    text2 = f.read()
print(text2)

successful pupil was so great and conspicuous for that day that on the spot every scholars heart was fired with a fresh ambition that often lasted a couple of weeks. It is possible that Toms mental stomach had never really hungered for one of those prizes, but unquestionably his entire being had for many a day longed for the glory and the eclat that came with it.
In due course the superintendent stood up in front of the pulpit, with a closed hymn-book in his hand and his forefinger inserted between its leaves, and commanded attention. When a Sunday-school superintendent makes his customary little speech, a hymn-book in the hand is as necessary as is the inevitable sheet of music in the hand of a singer who stands forward on the platform and sings a solo at a concertthough why, is a mystery: for neither the hymn-book nor the sheet of music is ever referred to by the sufferer. This superintendent was a slim creature of thirty-five, with a sandy goatee and short sandy hair; he wore a stif

In [21]:
key = bytes(a ^ ord(b) for (a, b) in zip(texts[1], text2))
print(key)

b'\x10\xdc\xdf7\xf1\xa9uU\x04U\xa2M6\x19<m\xaf\x12\x1a\'\xc6\ry\xfdH8\xb8\x86\xf7\xae\xf5#\xca\x1a\xc4\x16\x12s\x1d\xb9!1\x8c\x84\x1b\x80\xe3_\xfa\xe3\xd7\xcb\x88\x80\xda\x999 \x12\xdf4n\xa1\x9e2\xc2Y\xe4D \x82\xc5C\xf9&\xccT\x13Xw\x83\xcf\x04\x96\x89\xee\xe4V\xd7\xb2\xc8+K\x90\xc1\x8f02vYs\xc9\x14g\xeawx\xb7+\x7f\x1c\x8b\x89V\x14V\xff\xef\xb7,l\xfd\x17\xbf7\x862\xb8\x8a\x81+v\xb5\xd4\x15.\xf5Q\xf7!\xff\x9d\xb4\x10\xd8\x103\xa3\xe1\x93\x88\x8f\xa1\x11r\x93\xadj>h\xf4\xa0<W!\xcbe??\x03\x0b\x8d\xee\x8b\xb6\xcci\xac\xcd\xa5\x13\xf4#\rZ\xde4\x19$,\xcc\xb2\x138!\xa7Y\xf7\xa52\x9b\xf6\xb0T\xb62\xa3\x9f.5U\xd1\xcf\x1a\xcb\x8b \xe0-T\x9d\x1a)8,\x9a\x03\x9d!\xbb:\x96\xdf\xc0\x13z\xbc\x935\xab\x001_\xbe\xcbt\xdcK\x17\x83\x07\x85\x8f3|EO\x98\rN!\xab\x85\x14\xe1\x1c\xa5\xb3\x99k\x04\x19\xbf\x92\xa8\x84\x8c\xaf\xd7\xabb\x80}\xa6C\x08B]\x1f\xa5\xcd\xd3^XlS\xfcx`L\xd4(\xecsn\xd6\xcaB\x85\xe1\x02\xfe`k\x89\x08\xa2,\xafP+Z\xf7\x8e\xdd\xf7\xbf\x075\x81}nU\xea\x983\x0fT\xd5>!\x00\xab\xe2\xd22\xfe\xfe\xc2

Текст номер 1

In [24]:
text1 = "".join([chr(a ^ b) for (a, b) in zip(texts[0], key)])
with open("text1first5000sym.txt", 'w', encoding='utf-8') as f:
    f.write(text1)
print(text1)

in the fire, with her pretty hair fluttering in the wind and her eyes scorning me,often at such a time I would look towards those panels of black night in the wall which the wooden windows then were, and would fancy that I saw her just drawing her face away, and would believe that she had come at last. After that, when we went in to supper, the place and the meal
would have a more homely look than ever, and I would feel more ashamed of home than ever, in my own ungracious breast. Chapter XV. As I was getting too big for Mr. Wopsles great-aunts room, my education under that preposterous female terminated. Not, however, until Biddy had imparted to me everything she knew, from the little catalogue of prices, to a comic song she had once bought for a half-penny. Although the only coherent part of the latter piece of literature were the opening lines, When I went to Lunnon town sirs, Too rul loo rul Too rul loo rul Wasnt I done very brown sirs? Too rul loo rul Too rul loo rul still, in my d

Данный текст является отрывком из произведения Чарльза Диккенса "Большие надежды" (глава 14)

Текст номер 3

In [25]:
text3 = "".join([chr(a ^ b) for (a, b) in zip(texts[2], key)])
with open("text3first5000sym.txt", 'w', encoding='utf-8') as f:
    f.write(text3)
print(text3)

you? Yes. Well, there _is_ company there, I reckon. Better give it up. Give it up, and I just leaving this country forever! Give it up and maybe never have another chance. I tell you again, as Ive told you before, I dont care for her swagyou may have it. But her husband was rough on memany times he was rough on meand mainly he was the justice of the peace that juMged me for a vagrant. And that aint all. It aint a millionth part of it! He had me _horsewhipped_!horsewhipped in front of the jail, like a nigger!with all the town looking on! _Horsewhipped_!do you understand? He took advantage of me and died. But Ill take it out of _her_. Oh, dont kill her! Dont do that! Kill? Who said anything about killing? I would kill _him_ if he was here; but not her. When you want to get revenge on a woman you dont kill herbosh! you go for her looks. You slit her nostrilsyou notch her ears like a sow! By God, thats Keep your opinion to yourself! It will be safest for you. Ill tie her to the bed. If she

Данный текст так же был взят из произведения "Приключения Тома Сойера" (глава 29)